# Prepare S3 bucket to member pages 

1. Store list of scraped member pages as json file in S3 bucket. 
1. Get list of members to scrape-- "Following", "followed" people in members.  Before I do that, I need to restore member data lost in database.   


In [52]:
import pymongo 
from bs4 import BeautifulSoup as BS
import scraper
import boto3
import os 
import json
AWS_KEY = os.environ['AWS_ACCESS_KEY']
AWS_SECRET = os.environ['AWS_SECRET_ACCESS_KEY'] 

In [7]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
allrecipes_db = client.allrecipes

In [8]:
allrecipes_db.collection_names()

[u'community_link_pages',
 u'recipes',
 u'member_pages',
 u'members',
 u'webpages',
 u'healthy_recipes',
 u'about',
 u'community_page']

In [9]:
member_pages_coll = allrecipes_db.member_pages

In [10]:
 # Open s3 session with boto3
session = boto3.Session(aws_access_key_id=AWS_KEY, aws_secret_access_key=AWS_SECRET)
s3 = session.resource('s3')

In [11]:
# Select bucket 
mybucket = s3.Bucket('ohailolcat')

In [12]:
# Dump scraped member_IDs to a json file 
with open('members_list.json', 'w') as f: 
    json.dump({"members_in_mongodb":[page['member_ID'] 
         for page in member_pages_coll.find({'member_ID':{'$exists':True}},['member_ID'])]},f)


In [53]:
json.dump?

In [14]:
# Verify 
with open('members_list.json','r') as f: 
    mlist = json.load(f)
len(mlist.values()[0])

1340

In [55]:
# Upload to S3 
mybucket.upload_file('members_list.json', 'members/mongo_list')

In [56]:
s3_obj_list = session.client('s3').list_objects(Bucket='ohailolcat')

In [57]:
[a['Key'] for a in s3_obj_list['Contents'] if a['Key'].startswith('m')]

[u'members/mongo_list', u'members/to_scrape.json']

In [47]:
# Download from s3: 
session.client('s3').download_file('ohailolcat', 
                                   'members/mongo_list', 'members_list_s3.json')

In [59]:
session.client('s3').download_file('ohailolcat',
                                   'member_pages/9187834_pages.pkl', '9187834_pages.pkl')

In [61]:
import cPickle

In [63]:
pages = cPickle.load(open("9187834_pages.pkl", "r"))

In [64]:
pages.keys()

['favorites_page',
 'reviews_page',
 'followers_page',
 'madeit_page',
 'member_ID',
 'recipes_page',
 'following_page',
 'aboutme_page']

In [49]:
with open('members_list_s3.json', 'r') as f: 
    s = json.load(f)

In [51]:
len(s.values()[0])

1340